In [68]:
import sqlite3
import pandas as pd

In [76]:
conn = sqlite3.connect('proteins.db')
cursor = conn.cursor()

In [77]:
pd.read_sql_query("SELECT * FROM proteins;", conn)

,id,name,accession,function,domains,crispr_score,reference
0,5,BPK1,TGME49_253330,Needed for the development and infectivity of ...,"signal peptide, no transmembrane domain",-1.78,Buchholz et al. 2011; 2013
1,6,CST1,TGME49_064660,maintains cyst wall integrity and promotes bra...,"Transmembrane domain, no signal peptide",0.85,Guevara et al. 2020; Tomita et al. 2013; Zhang...
2,7,CST4,TGME49_261650,cyst wall proteins,"WD40 repeat-like domain, no signal peptide or ...",1.61,Tu et al. 2020
3,8,CST7,TgME49_258870,cyst wall proteins,"transmembrane domain, no signal peptide",1.32,Tu et al. 2020
4,9,CST8,TgME49_204340,cyst wall proteins,"signal peptide, transmembrane domains",1.6,Tu et al. 2020
...,...,...,...,...,...,...,...
75,80,PPMC3,TGME49_270320,Dephosphorylates secreted effector proteins wi...,"PP2C-class phosphatase domain, no signal pepti...",1.24,Mayoral et al. 2020
76,81,PSD1,TGME49_269920,Decarboxylates phosphatidylserine and needed f...,"Phosphatidylserine decarboxylase domain, signa...",0.52,Gupta et al. 2012
77,82,SFP1,TGGT1_289540,stranded structure formation within the PV,"four coiled-coil domains, no signal peptide or...",0.33,Young et al. 2020
78,83,WNG1,TGME49_304740,"Localized to the PV lumen, phosphorylates GRA ...",no signal peptide or transmembrane domain,1.1,Coffey et al. 2018; Beraki et al. 2019


In [73]:
def print_protein_info(df):
    """Prints the protein information from the DataFrame."""
    if df.empty:
        print("No data found.")
        return
    
    for index, row in df.iterrows():
        print(f"Name: {row['name']}")
        print(f"Accession: {row['accession']}")
        print(f"Function: {row['function']}")
        print(f"Domains: {row['domains']}")
        print(f"CRISPR Score: {row['crispr_score']}")
        print(f"References: {row['reference']}")
        print("--------------------------------------------------\n")


def query(field, value, match_type="exact"):
    """
    General-purpose query function for proteins table.

    Parameters:
        field (str): One of 'name', 'accession', 'function', 'domains', 'crispr_score', 'reference'.
        value (str or float): The value to search for.
        match_type (str): 'exact', 'like', or 'lt' (less than).

    Returns:
        pandas.DataFrame
    """
    valid_fields = ["name", "accession", "function", "domains", "crispr_score", "reference"]
    if field not in valid_fields:
        raise ValueError(f"Invalid field: {field}. Choose from: {', '.join(valid_fields)}")

    if match_type == "exact":
        query = f"SELECT * FROM proteins WHERE {field} = ?;"
        params = (value,)
    elif match_type == "like":
        query = f"SELECT * FROM proteins WHERE {field} LIKE ?;"
        params = (f"%{value}%",)
    elif match_type == "lt":
        query = f"SELECT * FROM proteins WHERE {field} < ?;"
        params = (value,)
    else:
        raise ValueError("match_type must be 'exact', 'like', or 'lt'")

    return pd.read_sql_query(query, conn, params=params)

query('function', 'escrt', match_type='like')

,id,name,accession,function,domains,crispr_score,reference
0,24,GRA14,TGGT1_239740,Uptake of host cytosolic proteins by co-op of ...,"transmembrane domain, no signal peptide",2.00,Bai et al. 2018; Rivera-Cuevas et al. 2021
1,67,GRA64,TGME49_202620,organizing the recruitment of ESCRT proteins a...,"signal peptide, transmembrane domains",0.94,Mayoral unpubl. data


In [47]:
def append_to_protein_by_name(name, updates: dict):
    """
    Appends new information to existing fields for a protein identified by its name.
    Does not commit the transaction — you must call conn.commit() manually.
    """
    # Fetch current values
    cursor.execute(f"SELECT {', '.join(updates.keys())} FROM proteins WHERE name = ?", (name,))
    result = cursor.fetchone()

    if result is None:
        print(f"No protein found with name '{name}'")
        return

    # Prepare new values
    new_values = []
    for i, (column, new_data) in enumerate(updates.items()):
        current_data = result[i]
        if current_data is None:
            updated_data = str(new_data)
        else:
            current_str = str(current_data).strip()
            new_str = str(new_data).strip()
            if new_str not in current_str:
                updated_data = current_str + "; " + new_str
            else:
                updated_data = current_str
        new_values.append(updated_data)

    # Build update query (no commit)
    set_clause = ", ".join([f"{key} = ?" for key in updates])
    query = f"UPDATE proteins SET {set_clause} WHERE name = ?;"
    cursor.execute(query, (*new_values, name))
    print(f"Prepared update for protein '{name}' — run conn.commit() to save changes.")


In [ ]:
append_to_protein_by_name("GRA01", {
    "function": "Involved in host cell invasion and immune evasion.",
    "domains": "Contains a GRA domain; also binds to host cell proteins.",
    "crispr_score": "0.08",
    "reference": "Smith et al., 2023; Doe et al., 2022"
})

df = pd.read_sql_query("SELECT * FROM proteins WHERE name = 'GRA01';", conn)
print_protein_info(df)

Prepared update for protein 'GRA01' — run conn.commit() to save changes.
Name: GRA01
Accession: TGME49_270250
Function: Potential function in suppression of host apoptosis. Localizes to the PV lumen; Involved in host cell invasion and immune evasion.
Domains: signal peptide, calcium binding domain, no transmembrane domain; Contains a GRA domain; also binds to host cell proteins.
CRISPR Score: -5.27; 0.08
References: (Cesbron-Delauw et al. 1989; Wu et al. 2017); Smith et al., 2023; Doe et al., 2022
--------------------------------------------------



In [64]:
conn.rollback()  # Rollback the transaction to avoid committing the changes

In [66]:
conn.commit()  # Commit the changes to the database

In [78]:
conn.close()  # Close the database connection